In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import numpy as np

equal weighted avg of past

In [6]:
thepast = torch.tensor([4,1,-2,-3])
#imagine this as activation vector, where each of the data values corres
# to info that we get from past
N = len(thepast)

weights = torch.ones(N) / N

print(f'The past: {thepast}')
print(f'weights (importance) of past: {weights}')
print(f'Sum over all weights: {sum(weights)}')

The past: tensor([ 4,  1, -2, -3])
weights (imp) of past: tensor([0.2500, 0.2500, 0.2500, 0.2500])
Sum over all weights: 1.0


In [8]:
thepresent = sum(thepast*weights)
print(f'The present (weighted sum of past): {thepresent}')

The present (weighted sum of past): 0.0


In [9]:
torch.ones(N)

tensor([1., 1., 1., 1.])

weighted avg of past

In [12]:
weights = torch.tensor([2,1,1,1])
print(f'Sum of weights: {sum(weights)}. Uh oh...')
#sum of weights is 5, not 1
                       

Sum of weights: 5. Uh oh...


In [16]:
linear_weights = weights/ sum(weights)
softmax_weights = torch.exp(weights)/sum(torch.exp(weights))

print(f'Scaled weights: {linear_weights}')
print(f'Their  sum: {sum(linear_weights)}')

print(f'softmax weights: {softmax_weights}')
print(f'Their  sum: {sum(softmax_weights)}')

#linear weights also have sum =1,
# but with softmax, sparsity is more...larger weights have pushed more and suppreses 
#smaller numbers


Scaled weights: tensor([0.4000, 0.2000, 0.2000, 0.2000])
Their  sum: 1.0
softmax weights: tensor([0.4754, 0.1749, 0.1749, 0.1749])
Their  sum: 1.0


In [17]:
thepresent_linear = sum(thepast*linear_weights)
thepresent_softmax = sum(thepast * softmax_weights)
print(f'The present (linear sum of past):  {thepresent_linear}')
print(f'The present (softmax sum of past):  {thepresent_softmax}')



The present (linear sum of past):  0.8000000715255737
The present (softmax sum of past):  1.2019567489624023


ignoring the future

In [19]:
thedata = torch.tensor([4,1,-2,-3,8,3,-1])
present_moment = 4 #that is value of 8, that means we
# want to use all of 4,1,-2,-3,8 and ignore all of the rest which is in future


N = len(thedata)
print(f'Past data: {thedata[:present_moment]}')
print(f'the present: {thedata[present_moment]}')
print(f'the future: {thedata[present_moment+1:]}')


Past data: tensor([ 4,  1, -2, -3])
the present: 8
the future: tensor([ 3, -1])


In [20]:
past_weights = torch.ones(N)
past_weights[present_moment+1:] = 0
past_weights
# this is unscaled weights (sum >1)
# so normalise this by dividing with sum in below cell

tensor([1., 1., 1., 1., 1., 0., 0.])

In [21]:
past_weights_linear = past_weights/torch.sum(past_weights)
print(f'Scaled weights: {past_weights_linear}')
print(f'Their sum: {sum(past_weights_linear)}')

Scaled weights: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000])
Their sum: 1.0


In [23]:
#softmax the wieghts with zeros
past_weights_softmax = torch.exp(past_weights) / torch.sum(torch.exp(past_weights))

print(f'Softmax weights: {past_weights_softmax}')
print(f'Their sum: {sum(past_weights_softmax)

#here the last two zeros are lifted up, due to chara of e(x)

Softmax weights: tensor([0.1743, 0.1743, 0.1743, 0.1743, 0.1743, 0.0641, 0.0641])
Their sum: 1.0


In [27]:
#e:g
torch.exp(torch.tensor([-1000]))

#exp of limit x tends to infi is 0

tensor([0.])

In [29]:
# recreaet wieghts for past, but setting future values to -inf
past_weights = torch.ones(N)
past_weights[present_moment+1:] = -torch.inf
# softmax
past_weights_softmax = torch.exp(past_weights) / torch.sum(torch.exp(past_weights))

print(f'Unscaled weights: {past_weights}')
print(f'Scaled weights: {past_weights_softmax}')
print(f'Their sum: {sum(past_weights_softmax)}')

Unscaled weights: tensor([1., 1., 1., 1., 1., -inf, -inf])
Scaled weights: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000])
Their sum: 1.0


steps towards the futre, looking back into past

In [31]:
# above was for single vector, now matrix

#rows are calculation steps, columns are time points
tril = torch.tril(torch.ones(9,9))
# torch.tril gives lower triangular elements, 
   # below and including the diagonal are preserved
   # above diagonal are set to 0 

tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [33]:
tril[tril==0] = -torch.inf # all the elements with value 0 is set to -inf
tril

tensor([[1., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., 1., -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., 1., 1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., 1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [34]:
#smax across columns within each row
tril_softmax = F.softmax(tril, dim=-1)
tril_softmax


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.0000],
        [0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111]])

In [37]:
for timepoint in range(tril.shape[0]):
    print(f'\nWeights for calculation at time point {timepoint}:')
    print(f'\t{tril_softmax[timepoint]}')


Weights for calculation at time point 0:
	tensor([1., 0., 0., 0., 0., 0., 0., 0., 0.])

Weights for calculation at time point 1:
	tensor([0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

Weights for calculation at time point 2:
	tensor([0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

Weights for calculation at time point 3:
	tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

Weights for calculation at time point 4:
	tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000])

Weights for calculation at time point 5:
	tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000, 0.0000])

Weights for calculation at time point 6:
	tensor([0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000, 0.0000])

Weights for calculation at time point 7:
	tensor([0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.0000])

Weights for calculation at time point 8:
	t

In [39]:
# till now,we have ones as starting values in the tril_softmax matrix
#IRL these values are the product of weights matrix and activations coming 
# from token embeddnings
# so it has +ve and -ve nos, not just ones

final demo with random activations


In [44]:
activations = torch.randn(N,N)
tril = torch.tril(torch.ones(N,N))

print('--- ORIGINAL ACITVATION:')
print(activations)


print('--- PAST WIEGHTING FACTOR:')
print(tril)

scaled_activations = activations*tril
scaled_activations[scaled_activations==0] = -torch.inf
print('\n--- SCALED PAST ACTIVATIONS:')
print(scaled_activations)

softmax_past = F.softmax(scaled_activations,dim=-1)
print('\n--- SOFTMAX PAST ACTIVATIONS:')
print(softmax_past)



--- ORIGINAL ACITVATION:
tensor([[-1.0040,  0.1279,  0.1113, -0.0473,  0.4878,  0.0382, -1.1958],
        [ 0.0504,  0.1063, -0.2585,  1.2443,  0.4743, -0.4885, -0.6628],
        [ 0.6822, -0.1675,  0.0359, -0.1218,  1.3053,  0.4093, -0.0220],
        [ 2.4790, -0.1282,  0.7991,  1.1209,  0.4441,  0.5420, -1.5590],
        [ 0.2744,  0.4507, -0.3256,  0.9051,  2.7350,  0.8098, -1.7626],
        [-0.4740,  0.0758,  0.0482, -0.2514,  0.2632,  0.3422,  0.7421],
        [ 0.9955, -0.3246,  0.6902, -0.2289,  0.9258, -0.2836,  0.7722]])
--- PAST WIEGHTING FACTOR:
tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.]])

--- SCALED PAST ACTIVATIONS:
tensor([[-1.0040,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0504,  0.1063,    -inf,    -inf,    -inf,    -inf,    

In [47]:
torch.sum(softmax_past,dim=-1) # sum over cols for each row is 1

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [49]:
# CREATION OF THIS MASK MATRIX IS FUSED INTO PyTorch
    #stay tuned